The model found during the base model finding phase. The future models will be compared to this.

In [2]:
# Organize data for pytorch DataLoader
import os
import shutil
import random
import torch

DATA_DIR = '../../dl2021-image-corpus-proj/'
# DATA_DIR = '/Users/hartih/Documents/School/Deep learning/Final_project/dl2021-image-corpus-proj/'
ANNOTATIONS_DIR = DATA_DIR + 'annotations/'
IMAGES_DIR = DATA_DIR + 'images/'

# New fodlers for train, test, and dev sets
TRAIN_DIR = DATA_DIR + 'train/'
DEV_DIR = DATA_DIR + 'dev/'
TEST_DIR = DATA_DIR + 'test/'

annotations = ["baby",
               "bird",
               "car",
               "clouds",
               "dog",
               "female",
               "flower",
               "male",
               "night",
               "people",
               "portrait",
               "river",
               "sea",
               "tree"]

In [3]:
# Create labels for images
image_file_names = os.listdir(IMAGES_DIR)
dict_labels = {}
for image_file_name in image_file_names:  # Initiate label tensors
    if os.path.isfile(IMAGES_DIR + image_file_name):
        dict_labels[image_file_name] = torch.zeros(14)
for i in range(len(annotations)):  # Fill label tensors with 1's if found in one of the annotations text files
    with open(ANNOTATIONS_DIR + annotations[i] + ".txt") as f:
        for row in f:
            row = "im" + row.strip() + ".jpg"
            dict_labels[row][i] = 1

In [3]:
dict_labels

{'im1.jpg': tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.]),
 'im10.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im100.jpg': tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.]),
 'im1000.jpg': tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im10000.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 'im10001.jpg': tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im10002.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.]),
 'im10003.jpg': tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im10004.jpg': tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.]),
 'im10005.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im10006.jpg': tensor([0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.]),
 'im10007.jpg': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'im10008.jpg': tensor([0.

In [5]:
# Split the data to train, test, and dev
os.makedirs(TRAIN_DIR)
os.makedirs(TEST_DIR)
os.makedirs(DEV_DIR)
for image_file_name in image_file_names:
    if os.path.isfile(IMAGES_DIR + image_file_name):
        division = random.randint(1, 4)
        if division == 1 or division == 2:
            shutil.copyfile(IMAGES_DIR + image_file_name, TRAIN_DIR + image_file_name)
        if division == 3:
            shutil.copyfile(IMAGES_DIR + image_file_name, DEV_DIR + image_file_name)
        if division == 4:
            shutil.copyfile(IMAGES_DIR + image_file_name, TEST_DIR + image_file_name)

In [4]:
print( len(os.listdir(TEST_DIR)) + len(os.listdir(TRAIN_DIR)) + len( os.listdir(DEV_DIR)) == len(os.listdir(IMAGES_DIR)) )
print(len(os.listdir(TEST_DIR)))
print(len(os.listdir(TRAIN_DIR)))
print(len(os.listdir(DEV_DIR)))
print(len(os.listdir(IMAGES_DIR)))


True
5020
9910
5070
20000


In [5]:
import torch.utils.data
from PIL import Image

# Enable creating train, test, and dev test datasets for PyTorch
class myDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir=IMAGES_DIR, transform=None):
        self.transform = transform
        self.root_dir = root_dir
        self.images = [root_dir + img for img in os.listdir(root_dir)]                
      
    def __len__(self):
        return len(self.images)       

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = img = Image.open(img_path)     
       
        if self.transform:
            img = self.transform(img)     

        return img, dict_labels[img_path.split("/")[-1]]

In [21]:
from torchvision import transforms

mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]


train_transform = transforms.Compose([
                                        transforms.Grayscale(num_output_channels=3),
                                        transforms.RandomCrop(128),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0),
                                        transforms.RandomAffine(degrees=20, translate=(0.1, 0.1), scale=(0.8, 1.2),
                                        shear=None, fill=(255, 255, 255)),
                                        transforms.RandomRotation(2),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=mean, std=std)])
test_transform = transforms.Compose([
                                        transforms.Grayscale(num_output_channels=3),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=mean, std=std)])

# Create datasets for CNN
train_set = myDataset(TRAIN_DIR, transform=train_transform)
test_set = myDataset(TEST_DIR, transform=train_transform)
dev_set = myDataset(DEV_DIR, transform=test_transform)

In [22]:
import torch 

# Calculate the amount of correctly predicted as well as total predictions done
def calc_correct(pred: torch.Tensor, target: torch.Tensor):
    pred = torch.sigmoid(pred)  # Since our neural network does not apply sigmoid
    correct_dict = {'tot': [0,0]}  # First number in value is correct ones, the second one is total amount
    correct_dict['tot_strict'] = [0,0] # All correct
    for i in range(len(pred)): # [100,14]
        all_correct = 0
        total = 0
        # estim_pred_array = []
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            # estim_pred_array.append(estim_pred)
            if annotations[j] not in correct_dict.keys():
                correct_dict[annotations[j]] = [0,0]
            correct_dict['tot'][1] += 1
            correct_dict[annotations[j]][1] += 1
            correct_dict['tot'][0] += int(estim_pred == target[i][j])
            correct_dict[annotations[j]][0] += int(estim_pred == target[i][j])
            all_correct += int(estim_pred == target[i][j])
            total += 1
        correct_dict['tot_strict'][1] += 1
        correct_dict['tot_strict'][0] += int(total==all_correct)
    # To-do how many pictures were entirely correct (accuracy)
    return correct_dict


In [23]:
def class_evaluation(pred: torch.Tensor, target: torch.Tensor):
    pred = torch.sigmoid(pred)
    
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    negative = 0
    positive = 0
    
    for i in range(len(pred)): # [100,14]
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            
            # Negative target values
            if target[i][j] == 0:
                negative += 1
                if estim_pred == 1:
                    false_positive += 1
                if estim_pred == 0:
                    true_negative += 1
            
            # Positive target values
            if target[i][j] == 1:
                positive += 1
                if estim_pred == 1:
                    true_positive += 1
                if estim_pred == 0:
                    false_negative += 1
    
    result = {"true_positive": true_positive,
            "false_positive": false_positive,
            "true_negative": true_negative,
            "false_negative": false_negative,
            "negative": negative,
            "positive": positive}
                    
    return result

In [24]:
def class_evaluation_by_annotation(pred: torch.Tensor, target: torch.Tensor):
    
    pred = torch.sigmoid(pred)
    
    # Initiate vslues for every annotation
    eval_dict = {}
    for a in range(len(annotations)):
        eval_dict[annotations[a]] = {"true_positive": 0,
                                        "false_positive": 0,
                                        "true_negative": 0,
                                        "false_negative": 0,
                                        "negative": 0,
                                        "positive": 0}
    
    for i in range(len(pred)):
        for j in range(len(pred[i])):
            estim_pred = 0 if float(pred[i][j]) < 0.5 else 1
            
            # Negative target values
            if target[i][j] == 0:
                eval_dict[annotations[j]]["negative"] += 1
                if estim_pred == 1:
                    eval_dict[annotations[j]]["false_positive"] += 1
                if estim_pred == 0:
                    eval_dict[annotations[j]]["true_negative"] += 1
            
            # Positive target values
            if target[i][j] == 1:
                eval_dict[annotations[j]]["positive"] += 1
                if estim_pred == 1:
                    eval_dict[annotations[j]]["true_positive"] += 1
                if estim_pred == 0:
                    eval_dict[annotations[j]]["false_negative"] += 1
                    
    return eval_dict

In [25]:
import torch
import torch.optim as optim
import torch.utils.data
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import transforms, datasets, models
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math


#--- hyperparameters ---
N_EPOCHS = 15
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_TEST = 100
LR = 0.001
WEIGHT_DECAY = 0.00
MOMENTUM = 0.0


#--- fixed constants ---
NUM_CLASSES = 14
NUM_CHANNELS = 3



# --- Dataset initialization ---

# We transform image files' contents to tensors
# Plus, we can add random transformations to the training data if we like
# Think on what kind of transformations may be meaningful for this data.
# Eg., horizontal-flip is definitely a bad idea for sign language data.
# You can use another transformation here if you find a better one.


# Create Pytorch data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE_TEST, shuffle=False)
dev_loader = torch.utils.data.DataLoader(dataset=dev_set, shuffle=False)

alexnet = models.alexnet(pretrained=True)

#--- model ---
class CNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(NUM_CHANNELS, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(64, 128, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128, 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.flatten = nn.Flatten()
        self.lin = nn.Sequential(
            nn.Dropout(p=0.15),
            nn.Linear(65536, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.15),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.lin(x)
        return x



#--- set up ---
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = CNN().to(device)


optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)
loss_function = nn.BCEWithLogitsLoss()

dev_loss = math.inf
dev_losses = []
dev_accuracies = []
stop_early = False

#--- set up ---
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = CNN().to(device)


optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
loss_function = nn.BCEWithLogitsLoss()

dev_loss = math.inf
dev_losses = []
dev_accuracies = []
num_to_early_stop = 5  # Compares the current loss to loss we had before, this defines how far back we look  
stop_early = False
num_same_dev = 1

#--- training ---
for epoch in range(N_EPOCHS):
    if stop_early:
        break
    train_loss = 0
    train_correct = {annotation: [0,0] for annotation in annotations}
    train_correct['tot'] = [0,0]
    train_correct['tot_strict'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}
    total = 0

    for batch_num, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += len(data)
        train_loss += loss.item()
        new_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_correct[annotation]
            train_correct[annotation][0] += new[0]
            train_correct[annotation][1] += new[1]
        train_correct['tot'][0] += new_correct['tot'][0]
        train_correct['tot'][1] += new_correct['tot'][1]
        train_correct['tot_strict'][0] += new_correct['tot_strict'][0]
        train_correct['tot_strict'][1] += new_correct['tot_strict'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]

        print("------------------------")
        print('Training: Epoch %d - Batch %d/%d: Loss: %.4f | Train Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
              (epoch+1, batch_num+1, len(train_loader), train_loss / (batch_num + 1), 
               100. * train_correct['tot'][0] / train_correct['tot'][1], train_correct['tot'][0], train_correct['tot'][1],
               100. * train_correct['tot_strict'][0] / train_correct['tot_strict'][1], train_correct['tot_strict'][0], train_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")

    cur_dev_loss = 0
    dev_correct = {annotation: [0,0] for annotation in annotations}
    dev_correct['tot'] = [0,0]
    dev_correct['tot_strict'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

    with torch.no_grad():
        for batch_num, (data, target) in enumerate(dev_loader):
            data, target = data.to(device), target.to(device)
            pred = model(data)
            loss = loss_function(pred, target)

            cur_dev_loss += loss.item()
            new_dev_correct = calc_correct(pred, target)
            for annotation in annotations:
                new = new_dev_correct[annotation]
                dev_correct[annotation][0] += new[0]
                dev_correct[annotation][1] += new[1]
            dev_correct['tot'][0] += new_dev_correct['tot'][0]
            dev_correct['tot'][1] += new_dev_correct['tot'][1]
            dev_correct['tot_strict'][0] += new_dev_correct['tot_strict'][0]
            dev_correct['tot_strict'][1] += new_dev_correct['tot_strict'][1]
            
            evaluations = class_evaluation(pred, target)
            evaluation["true_positive"] += evaluations["true_positive"]
            evaluation["false_positive"] += evaluations["false_positive"]
            evaluation["true_negative"] += evaluations["true_negative"]
            evaluation["false_negative"] += evaluations["false_negative"]
            evaluation["positive"] += evaluations["positive"]
            evaluation["negative"] += evaluations["negative"]

        current_loss = cur_dev_loss / (len(dev_loader) + 1)
        dev_losses.append(current_loss)
        current_accuracy = {annotation: 100. * dev_correct[annotation][0] / dev_correct[annotation][1]  for annotation in annotations}  # Accuracies for all classes 
        current_accuracy['tot'] = 100. * dev_correct['tot'][0] / dev_correct['tot'][1]
        current_accuracy['tot_strict'] = 100. * dev_correct['tot_strict'][0] / dev_correct['tot_strict'][1]
        dev_accuracies.append(current_accuracy)

        # if current_loss <= dev_loss:
        #     if current_loss == dev_loss:
        #         num_same_dev += 1
        #     else:
        #         num_same_dev = 1
        #     dev_loss = current_loss
        #     if num_same_dev >= 5:  # If dev does not improve, stop doing epochs
        #         stop_early = True
        # else:
        #     stop_early = True
        # if len(dev_losses) >= 5 and dev_losses[-5]<dev_losses[-1]:
        #     stop_early = True

        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Dev Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
            (batch_num+1, len(dev_loader), cur_dev_loss / (len(dev_loader) + 1), 
            100. * dev_correct['tot'][0] / dev_correct['tot'][1], dev_correct['tot'][0], dev_correct['tot'][1],
            100. * dev_correct['tot_strict'][0] / dev_correct['tot_strict'][1], dev_correct['tot_strict'][0], dev_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")


c:\Users\juhan\anaconda3\envs\pytorch3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\juhan\anaconda3\envs\pytorch3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


------------------------
Training: Epoch 1 - Batch 1/100: Loss: 0.6987 | Train Acc: 36.571% (512/1400) | Strict Acc: 0.000% (0/100)
True positive rate: 56.604% (60/106)
False negative rate: 43.396% (46/106)
True negative rate: 34.930% (452/1294)
False positive rate: 65.070% (842/1294)
------------------------
------------------------
Training: Epoch 1 - Batch 2/100: Loss: 0.5036 | Train Acc: 63.214% (1770/2800) | Strict Acc: 1.000% (2/200)
True positive rate: 43.333% (91/210)
False negative rate: 56.667% (119/210)
True negative rate: 64.826% (1679/2590)
False positive rate: 35.174% (911/2590)
------------------------
------------------------
Training: Epoch 1 - Batch 3/100: Loss: 1.0524 | Train Acc: 73.119% (3071/4200) | Strict Acc: 17.000% (51/300)
True positive rate: 29.450% (91/309)
False negative rate: 70.550% (218/309)
True negative rate: 76.587% (2980/3891)
False positive rate: 23.413% (911/3891)
------------------------
------------------------
Training: Epoch 1 - Batch 4/100: L

In [26]:
#--- test ---
model.eval()

test_loss = 0
test_correct = {annotation: [0,0] for annotation in annotations}
test_correct['tot'] = [0,0]
test_correct['tot_strict'] = [0,0]
evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

evaluation_by_annotation = {}
for a in range(len(annotations)):
    evaluation_by_annotation[annotations[a]] = {"true_positive": 0,
                                                "false_positive": 0,
                                                "true_negative": 0,
                                                "false_negative": 0,
                                                "negative": 0,
                                                "positive": 0}

with torch.no_grad():
    for batch_num, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        test_loss += loss.item()
        new_test_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_test_correct[annotation]
            test_correct[annotation][0] += new[0]
            test_correct[annotation][1] += new[1]
        test_correct['tot'][0] += new_test_correct['tot'][0]
        test_correct['tot'][1] += new_test_correct['tot'][1]
        test_correct['tot_strict'][0] += new_test_correct['tot_strict'][0]
        test_correct['tot_strict'][1] += new_test_correct['tot_strict'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]
        
        evaluations_by_annotation = class_evaluation_by_annotation(pred, target)
        print(evaluation_by_annotation)
        
        for a in range(len(annotations)):
            evaluation_by_annotation[annotations[a]]["true_positive"] += evaluations_by_annotation[annotations[a]]["true_positive"]
            evaluation_by_annotation[annotations[a]]["false_positive"] += evaluations_by_annotation[annotations[a]]["false_positive"]
            evaluation_by_annotation[annotations[a]]["true_negative"] += evaluations_by_annotation[annotations[a]]["true_negative"]
            evaluation_by_annotation[annotations[a]]["false_negative"] += evaluations_by_annotation[annotations[a]]["false_negative"]
            evaluation_by_annotation[annotations[a]]["positive"] += evaluations_by_annotation[annotations[a]]["positive"]
            evaluation_by_annotation[annotations[a]]["negative"] += evaluations_by_annotation[annotations[a]]["negative"]
        
        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Test Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
              (batch_num+1, len(test_loader), test_loss / (batch_num + 1), 
               100. * test_correct['tot'][0] / test_correct['tot'][1], test_correct['tot'][0], test_correct['tot'][1],
               100. * test_correct['tot_strict'][0] / test_correct['tot_strict'][1], test_correct['tot_strict'][0], test_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")
        
print(dev_losses)
print(dev_accuracies)

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'male': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'night': {'true_pos

In [29]:
for i in evaluation_by_annotation:
    print("------------------------")
    print(i + ":")
    print("")
    print("positive: " + str(evaluation_by_annotation[i]["positive"]))
    print("true_positive: " + str(evaluation_by_annotation[i]["true_positive"]))
    print("false_negative: " + str(evaluation_by_annotation[i]["false_negative"]))
    print("")
    print("negative: " + str(evaluation_by_annotation[i]["negative"]))
    print("true_negative: " + str(evaluation_by_annotation[i]["true_negative"]))
    print("false_positive: " + str(evaluation_by_annotation[i]["false_positive"]))
        


------------------------
baby:

positive: 22
true_positive: 0
false_negative: 22

negative: 4998
true_negative: 4998
false_positive: 0
------------------------
bird:

positive: 87
true_positive: 0
false_negative: 87

negative: 4933
true_negative: 4933
false_positive: 0
------------------------
car:

positive: 103
true_positive: 0
false_negative: 103

negative: 4917
true_negative: 4917
false_positive: 0
------------------------
clouds:

positive: 255
true_positive: 3
false_negative: 252

negative: 4765
true_negative: 4757
false_positive: 8
------------------------
dog:

positive: 104
true_positive: 0
false_negative: 104

negative: 4916
true_negative: 4916
false_positive: 0
------------------------
female:

positive: 774
true_positive: 6
false_negative: 768

negative: 4246
true_negative: 4245
false_positive: 1
------------------------
flower:

positive: 191
true_positive: 0
false_negative: 191

negative: 4829
true_negative: 4829
false_positive: 0
------------------------
male:

positive:

In [31]:
model.train()

#--- training ---
for epoch in range(N_EPOCHS, N_EPOCHS+3):
    if stop_early:
        break
    train_loss = 0
    train_correct = {annotation: [0,0] for annotation in annotations}
    train_correct['tot'] = [0,0]
    train_correct['tot_strict'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}
    total = 0

    for batch_num, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += len(data)
        train_loss += loss.item()
        new_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_correct[annotation]
            train_correct[annotation][0] += new[0]
            train_correct[annotation][1] += new[1]
        train_correct['tot'][0] += new_correct['tot'][0]
        train_correct['tot'][1] += new_correct['tot'][1]
        train_correct['tot_strict'][0] += new_correct['tot_strict'][0]
        train_correct['tot_strict'][1] += new_correct['tot_strict'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]

        print("------------------------")
        print('Training: Epoch %d - Batch %d/%d: Loss: %.4f | Train Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
              (epoch+1, batch_num+1, len(train_loader), train_loss / (batch_num + 1), 
               100. * train_correct['tot'][0] / train_correct['tot'][1], train_correct['tot'][0], train_correct['tot'][1],
               100. * train_correct['tot_strict'][0] / train_correct['tot_strict'][1], train_correct['tot_strict'][0], train_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")

    cur_dev_loss = 0
    dev_correct = {annotation: [0,0] for annotation in annotations}
    dev_correct['tot'] = [0,0]
    dev_correct['tot_strict'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

    with torch.no_grad():
        for batch_num, (data, target) in enumerate(dev_loader):
            data, target = data.to(device), target.to(device)
            pred = model(data)
            loss = loss_function(pred, target)

            cur_dev_loss += loss.item()
            new_dev_correct = calc_correct(pred, target)
            for annotation in annotations:
                new = new_dev_correct[annotation]
                dev_correct[annotation][0] += new[0]
                dev_correct[annotation][1] += new[1]
            dev_correct['tot'][0] += new_dev_correct['tot'][0]
            dev_correct['tot'][1] += new_dev_correct['tot'][1]
            dev_correct['tot_strict'][0] += new_dev_correct['tot_strict'][0]
            dev_correct['tot_strict'][1] += new_dev_correct['tot_strict'][1]
            
            evaluations = class_evaluation(pred, target)
            evaluation["true_positive"] += evaluations["true_positive"]
            evaluation["false_positive"] += evaluations["false_positive"]
            evaluation["true_negative"] += evaluations["true_negative"]
            evaluation["false_negative"] += evaluations["false_negative"]
            evaluation["positive"] += evaluations["positive"]
            evaluation["negative"] += evaluations["negative"]

        current_loss = cur_dev_loss / (len(dev_loader) + 1)
        dev_losses.append(current_loss)
        current_accuracy = {annotation: 100. * dev_correct[annotation][0] / dev_correct[annotation][1]  for annotation in annotations}  # Accuracies for all classes 
        current_accuracy['tot'] = 100. * dev_correct['tot'][0] / dev_correct['tot'][1]
        current_accuracy['tot_strict'] = 100. * dev_correct['tot_strict'][0] / dev_correct['tot_strict'][1]
        dev_accuracies.append(current_accuracy)

        # if current_loss <= dev_loss:
        #     if current_loss == dev_loss:
        #         num_same_dev += 1
        #     else:
        #         num_same_dev = 1
        #     dev_loss = current_loss
        #     if num_same_dev >= 5:  # If dev does not improve, stop doing epochs
        #         stop_early = True
        # else:
        #     stop_early = True
        # if len(dev_losses) >= 5 and dev_losses[-5]<dev_losses[-1]:
        #     stop_early = True

        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Dev Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
            (batch_num+1, len(dev_loader), cur_dev_loss / (len(dev_loader) + 1), 
            100. * dev_correct['tot'][0] / dev_correct['tot'][1], dev_correct['tot'][0], dev_correct['tot'][1],
            100. * dev_correct['tot_strict'][0] / dev_correct['tot_strict'][1], dev_correct['tot_strict'][0], dev_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")

------------------------
Training: Epoch 16 - Batch 1/100: Loss: 0.1832 | Train Acc: 93.143% (1304/1400) | Strict Acc: 53.000% (53/100)
True positive rate: 0.000% (0/94)
False negative rate: 100.000% (94/94)
True negative rate: 99.847% (1304/1306)
False positive rate: 0.153% (2/1306)
------------------------
------------------------
Training: Epoch 16 - Batch 2/100: Loss: 0.2041 | Train Acc: 92.500% (2590/2800) | Strict Acc: 50.000% (100/200)
True positive rate: 0.985% (2/203)
False negative rate: 99.015% (201/203)
True negative rate: 99.653% (2588/2597)
False positive rate: 0.347% (9/2597)
------------------------
------------------------
Training: Epoch 16 - Batch 3/100: Loss: 0.2013 | Train Acc: 92.810% (3898/4200) | Strict Acc: 49.000% (147/300)
True positive rate: 2.048% (6/293)
False negative rate: 97.952% (287/293)
True negative rate: 99.616% (3892/3907)
False positive rate: 0.384% (15/3907)
------------------------
------------------------
Training: Epoch 16 - Batch 4/100: Loss

In [32]:
#--- test ---
model.eval()

test_loss = 0
test_correct = {annotation: [0,0] for annotation in annotations}
test_correct['tot'] = [0,0]
test_correct['tot_strict'] = [0,0]
evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

evaluation_by_annotation = {}
for a in range(len(annotations)):
    evaluation_by_annotation[annotations[a]] = {"true_positive": 0,
                                                "false_positive": 0,
                                                "true_negative": 0,
                                                "false_negative": 0,
                                                "negative": 0,
                                                "positive": 0}

with torch.no_grad():
    for batch_num, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        test_loss += loss.item()
        new_test_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_test_correct[annotation]
            test_correct[annotation][0] += new[0]
            test_correct[annotation][1] += new[1]
        test_correct['tot'][0] += new_test_correct['tot'][0]
        test_correct['tot'][1] += new_test_correct['tot'][1]
        test_correct['tot_strict'][0] += new_test_correct['tot_strict'][0]
        test_correct['tot_strict'][1] += new_test_correct['tot_strict'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]
        
        evaluations_by_annotation = class_evaluation_by_annotation(pred, target)
        print(evaluation_by_annotation)
        
        for a in range(len(annotations)):
            evaluation_by_annotation[annotations[a]]["true_positive"] += evaluations_by_annotation[annotations[a]]["true_positive"]
            evaluation_by_annotation[annotations[a]]["false_positive"] += evaluations_by_annotation[annotations[a]]["false_positive"]
            evaluation_by_annotation[annotations[a]]["true_negative"] += evaluations_by_annotation[annotations[a]]["true_negative"]
            evaluation_by_annotation[annotations[a]]["false_negative"] += evaluations_by_annotation[annotations[a]]["false_negative"]
            evaluation_by_annotation[annotations[a]]["positive"] += evaluations_by_annotation[annotations[a]]["positive"]
            evaluation_by_annotation[annotations[a]]["negative"] += evaluations_by_annotation[annotations[a]]["negative"]
        
        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Test Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
              (batch_num+1, len(test_loader), test_loss / (batch_num + 1), 
               100. * test_correct['tot'][0] / test_correct['tot'][1], test_correct['tot'][0], test_correct['tot'][1],
               100. * test_correct['tot_strict'][0] / test_correct['tot_strict'][1], test_correct['tot_strict'][0], test_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")
        
print(dev_losses)
print(dev_accuracies)

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'male': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'night': {'true_pos

In [33]:
for i in evaluation_by_annotation:
    print("------------------------")
    print(i + ":")
    print("")
    print("positive: " + str(evaluation_by_annotation[i]["positive"]))
    print("true_positive: " + str(evaluation_by_annotation[i]["true_positive"]))
    print("false_negative: " + str(evaluation_by_annotation[i]["false_negative"]))
    print("")
    print("negative: " + str(evaluation_by_annotation[i]["negative"]))
    print("true_negative: " + str(evaluation_by_annotation[i]["true_negative"]))
    print("false_positive: " + str(evaluation_by_annotation[i]["false_positive"]))

------------------------
baby:

positive: 22
true_positive: 0
false_negative: 22

negative: 4998
true_negative: 4998
false_positive: 0
------------------------
bird:

positive: 87
true_positive: 0
false_negative: 87

negative: 4933
true_negative: 4933
false_positive: 0
------------------------
car:

positive: 103
true_positive: 0
false_negative: 103

negative: 4917
true_negative: 4917
false_positive: 0
------------------------
clouds:

positive: 255
true_positive: 1
false_negative: 254

negative: 4765
true_negative: 4765
false_positive: 0
------------------------
dog:

positive: 104
true_positive: 0
false_negative: 104

negative: 4916
true_negative: 4916
false_positive: 0
------------------------
female:

positive: 774
true_positive: 23
false_negative: 751

negative: 4246
true_negative: 4238
false_positive: 8
------------------------
flower:

positive: 191
true_positive: 0
false_negative: 191

negative: 4829
true_negative: 4829
false_positive: 0
------------------------
male:

positive

In [34]:
model.train()

#--- training ---
for epoch in range(N_EPOCHS+3, N_EPOCHS+5):
    if stop_early:
        break
    train_loss = 0
    train_correct = {annotation: [0,0] for annotation in annotations}
    train_correct['tot'] = [0,0]
    train_correct['tot_strict'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}
    total = 0

    for batch_num, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += len(data)
        train_loss += loss.item()
        new_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_correct[annotation]
            train_correct[annotation][0] += new[0]
            train_correct[annotation][1] += new[1]
        train_correct['tot'][0] += new_correct['tot'][0]
        train_correct['tot'][1] += new_correct['tot'][1]
        train_correct['tot_strict'][0] += new_correct['tot_strict'][0]
        train_correct['tot_strict'][1] += new_correct['tot_strict'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]

        print("------------------------")
        print('Training: Epoch %d - Batch %d/%d: Loss: %.4f | Train Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
              (epoch+1, batch_num+1, len(train_loader), train_loss / (batch_num + 1), 
               100. * train_correct['tot'][0] / train_correct['tot'][1], train_correct['tot'][0], train_correct['tot'][1],
               100. * train_correct['tot_strict'][0] / train_correct['tot_strict'][1], train_correct['tot_strict'][0], train_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")

    cur_dev_loss = 0
    dev_correct = {annotation: [0,0] for annotation in annotations}
    dev_correct['tot'] = [0,0]
    dev_correct['tot_strict'] = [0,0]
    evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

    with torch.no_grad():
        for batch_num, (data, target) in enumerate(dev_loader):
            data, target = data.to(device), target.to(device)
            pred = model(data)
            loss = loss_function(pred, target)

            cur_dev_loss += loss.item()
            new_dev_correct = calc_correct(pred, target)
            for annotation in annotations:
                new = new_dev_correct[annotation]
                dev_correct[annotation][0] += new[0]
                dev_correct[annotation][1] += new[1]
            dev_correct['tot'][0] += new_dev_correct['tot'][0]
            dev_correct['tot'][1] += new_dev_correct['tot'][1]
            dev_correct['tot_strict'][0] += new_dev_correct['tot_strict'][0]
            dev_correct['tot_strict'][1] += new_dev_correct['tot_strict'][1]
            
            evaluations = class_evaluation(pred, target)
            evaluation["true_positive"] += evaluations["true_positive"]
            evaluation["false_positive"] += evaluations["false_positive"]
            evaluation["true_negative"] += evaluations["true_negative"]
            evaluation["false_negative"] += evaluations["false_negative"]
            evaluation["positive"] += evaluations["positive"]
            evaluation["negative"] += evaluations["negative"]

        current_loss = cur_dev_loss / (len(dev_loader) + 1)
        dev_losses.append(current_loss)
        current_accuracy = {annotation: 100. * dev_correct[annotation][0] / dev_correct[annotation][1]  for annotation in annotations}  # Accuracies for all classes 
        current_accuracy['tot'] = 100. * dev_correct['tot'][0] / dev_correct['tot'][1]
        current_accuracy['tot_strict'] = 100. * dev_correct['tot_strict'][0] / dev_correct['tot_strict'][1]
        dev_accuracies.append(current_accuracy)

        # if current_loss <= dev_loss:
        #     if current_loss == dev_loss:
        #         num_same_dev += 1
        #     else:
        #         num_same_dev = 1
        #     dev_loss = current_loss
        #     if num_same_dev >= 5:  # If dev does not improve, stop doing epochs
        #         stop_early = True
        # else:
        #     stop_early = True
        # if len(dev_losses) >= 5 and dev_losses[-5]<dev_losses[-1]:
        #     stop_early = True

        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Dev Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
            (batch_num+1, len(dev_loader), cur_dev_loss / (len(dev_loader) + 1), 
            100. * dev_correct['tot'][0] / dev_correct['tot'][1], dev_correct['tot'][0], dev_correct['tot'][1],
            100. * dev_correct['tot_strict'][0] / dev_correct['tot_strict'][1], dev_correct['tot_strict'][0], dev_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")

------------------------
Training: Epoch 19 - Batch 1/100: Loss: 0.1659 | Train Acc: 94.214% (1319/1400) | Strict Acc: 53.000% (53/100)
True positive rate: 7.229% (6/83)
False negative rate: 92.771% (77/83)
True negative rate: 99.696% (1313/1317)
False positive rate: 0.304% (4/1317)
------------------------
------------------------
Training: Epoch 19 - Batch 2/100: Loss: 0.1914 | Train Acc: 93.107% (2607/2800) | Strict Acc: 48.000% (96/200)
True positive rate: 7.000% (14/200)
False negative rate: 93.000% (186/200)
True negative rate: 99.731% (2593/2600)
False positive rate: 0.269% (7/2600)
------------------------
------------------------
Training: Epoch 19 - Batch 3/100: Loss: 0.1873 | Train Acc: 93.405% (3923/4200) | Strict Acc: 49.333% (148/300)
True positive rate: 7.292% (21/288)
False negative rate: 92.708% (267/288)
True negative rate: 99.744% (3902/3912)
False positive rate: 0.256% (10/3912)
------------------------
------------------------
Training: Epoch 19 - Batch 4/100: Loss

In [35]:
#--- test ---
model.eval()

test_loss = 0
test_correct = {annotation: [0,0] for annotation in annotations}
test_correct['tot'] = [0,0]
test_correct['tot_strict'] = [0,0]
evaluation = {"true_positive": 0,
                    "false_positive": 0,
                    "true_negative": 0,
                    "false_negative": 0,
                    "negative": 0,
                    "positive": 0}

evaluation_by_annotation = {}
for a in range(len(annotations)):
    evaluation_by_annotation[annotations[a]] = {"true_positive": 0,
                                                "false_positive": 0,
                                                "true_negative": 0,
                                                "false_negative": 0,
                                                "negative": 0,
                                                "positive": 0}

with torch.no_grad():
    for batch_num, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        pred = model(data)
        loss = loss_function(pred, target)

        test_loss += loss.item()
        new_test_correct = calc_correct(pred, target)
        for annotation in annotations:
            new = new_test_correct[annotation]
            test_correct[annotation][0] += new[0]
            test_correct[annotation][1] += new[1]
        test_correct['tot'][0] += new_test_correct['tot'][0]
        test_correct['tot'][1] += new_test_correct['tot'][1]
        test_correct['tot_strict'][0] += new_test_correct['tot_strict'][0]
        test_correct['tot_strict'][1] += new_test_correct['tot_strict'][1]
        
        evaluations = class_evaluation(pred, target)
        evaluation["true_positive"] += evaluations["true_positive"]
        evaluation["false_positive"] += evaluations["false_positive"]
        evaluation["true_negative"] += evaluations["true_negative"]
        evaluation["false_negative"] += evaluations["false_negative"]
        evaluation["positive"] += evaluations["positive"]
        evaluation["negative"] += evaluations["negative"]
        
        evaluations_by_annotation = class_evaluation_by_annotation(pred, target)
        print(evaluation_by_annotation)
        
        for a in range(len(annotations)):
            evaluation_by_annotation[annotations[a]]["true_positive"] += evaluations_by_annotation[annotations[a]]["true_positive"]
            evaluation_by_annotation[annotations[a]]["false_positive"] += evaluations_by_annotation[annotations[a]]["false_positive"]
            evaluation_by_annotation[annotations[a]]["true_negative"] += evaluations_by_annotation[annotations[a]]["true_negative"]
            evaluation_by_annotation[annotations[a]]["false_negative"] += evaluations_by_annotation[annotations[a]]["false_negative"]
            evaluation_by_annotation[annotations[a]]["positive"] += evaluations_by_annotation[annotations[a]]["positive"]
            evaluation_by_annotation[annotations[a]]["negative"] += evaluations_by_annotation[annotations[a]]["negative"]
        
        print("------------------------")
        print('Evaluating: Batch %d/%d: Loss: %.4f | Test Acc: %.3f%% (%d/%d) | Strict Acc: %.3f%% (%d/%d)' % 
              (batch_num+1, len(test_loader), test_loss / (batch_num + 1), 
               100. * test_correct['tot'][0] / test_correct['tot'][1], test_correct['tot'][0], test_correct['tot'][1],
               100. * test_correct['tot_strict'][0] / test_correct['tot_strict'][1], test_correct['tot_strict'][0], test_correct['tot_strict'][1]))
        print('True positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_positive"] / evaluation["positive"], evaluation["true_positive"], evaluation["positive"]) )
        print('False negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_negative"] / evaluation["positive"], evaluation["false_negative"], evaluation["positive"]) )
        print('True negative rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["true_negative"] / evaluation["negative"], evaluation["true_negative"], evaluation["negative"]) )
        print('False positive rate: %.3f%% (%d/%d)' % 
              (100. * evaluation["false_positive"] / evaluation["negative"], evaluation["false_positive"], evaluation["negative"]) )
        print("------------------------")
        
print(dev_losses)
print(dev_accuracies)

{'baby': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'bird': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'car': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'clouds': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'dog': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'female': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'flower': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'male': {'true_positive': 0, 'false_positive': 0, 'true_negative': 0, 'false_negative': 0, 'negative': 0, 'positive': 0}, 'night': {'true_pos

In [36]:
for i in evaluation_by_annotation:
    print("------------------------")
    print(i + ":")
    print("")
    print("positive: " + str(evaluation_by_annotation[i]["positive"]))
    print("true_positive: " + str(evaluation_by_annotation[i]["true_positive"]))
    print("false_negative: " + str(evaluation_by_annotation[i]["false_negative"]))
    print("")
    print("negative: " + str(evaluation_by_annotation[i]["negative"]))
    print("true_negative: " + str(evaluation_by_annotation[i]["true_negative"]))
    print("false_positive: " + str(evaluation_by_annotation[i]["false_positive"]))

------------------------
baby:

positive: 22
true_positive: 0
false_negative: 22

negative: 4998
true_negative: 4998
false_positive: 0
------------------------
bird:

positive: 87
true_positive: 0
false_negative: 87

negative: 4933
true_negative: 4933
false_positive: 0
------------------------
car:

positive: 103
true_positive: 0
false_negative: 103

negative: 4917
true_negative: 4917
false_positive: 0
------------------------
clouds:

positive: 255
true_positive: 0
false_negative: 255

negative: 4765
true_negative: 4765
false_positive: 0
------------------------
dog:

positive: 104
true_positive: 0
false_negative: 104

negative: 4916
true_negative: 4916
false_positive: 0
------------------------
female:

positive: 774
true_positive: 117
false_negative: 657

negative: 4246
true_negative: 4164
false_positive: 82
------------------------
flower:

positive: 191
true_positive: 0
false_negative: 191

negative: 4829
true_negative: 4829
false_positive: 0
------------------------
male:

positi

In [37]:
PATH = DATA_DIR + "cnn_base.pt"

torch.save(model.state_dict(), PATH)